In [1]:
import cv2
import numpy as np

In [2]:
import mediapipe as mp

In [3]:
mpDraw = mp.solutions.drawing_utils

In [4]:
mpHands = mp.solutions.hands

In [5]:
hands = mpHands.Hands(
                       static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.5 
                     )

In [6]:
tipsID = [4,8,12,16,20]

In [7]:
cap = cv2.VideoCapture(0)
cap.set(3,1135)
cap.set(4,720)

True

In [8]:
header = cv2.imread('paint.png')

In [9]:
draw_color = (255,0,0)

In [10]:
def findHand(results ,img) :
    
    if results.multi_hand_landmarks :
        for Handlms in results.multi_hand_landmarks :
            mpDraw.draw_landmarks(img ,Handlms ,mpHands.HAND_CONNECTIONS)
    
    return img

In [11]:
def findPosition(results ,img ):
    lmList = []
    h,w = img.shape[:2]
    
    if results.multi_hand_landmarks :
        for Handlms in results.multi_hand_landmarks :
            for id,lm in enumerate(Handlms.landmark) :
                cx , cy = int(lm.x * w) , int(lm.y * h)
                lmList.append([id,cx,cy])
    
    return lmList

In [12]:
def fingersUP(lmList):
    fingers = []
    
    # thumb
    if  lmList[tipsID[0]][1]<lmList[tipsID[0]-1][1] :
        fingers.append(1)
    else:
        fingers.append(0)
        
    
    # fingers
    for id in range(1,5):
        if  lmList[tipsID[id]][2]<lmList[tipsID[id]-2][2] :
            fingers.append(1)
        else:
            fingers.append(0)
    
    return fingers

In [13]:
xp,yp = 0,0
eraser_thickness = 40
brush_thickness = 20

In [14]:
img_canvas = np.zeros((720,1280,3) ,dtype='uint8')

In [15]:
while True :
    _ , img = cap.read()
    h,w = img.shape[:2]
    
    # we have to flip the image ,because if we move to left it will draw on right
    img = cv2.flip(img ,1)
    
    imgRGB = cv2.cvtColor(img ,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    img = findHand(results ,img)
    lmList = findPosition(results ,img)
    
    if len(lmList)!=0 :
        
        # index finger
        x1 ,y1 = lmList[8][1:]
        # middle finger
        x2 ,y2 = lmList[12][1:]
        
        # Check which finger is up
        fingers = fingersUP(lmList)
        
        # selection mode
        if fingers[1]==1 and fingers[2]==1:
            xp,yp = 0,0
            
            if y1<141 :
                
                # color 1
                if 64<x1<180:
                    draw_color = (0,0,255)
                    
                # color 2
                elif 280<x1<378:
                    draw_color = (204, 51, 51)
                    
                # color 3
                elif 473<x1<570:
                    draw_color = (204, 255 ,102)
                    
                # color 4
                elif 678<x1<782:
                    draw_color = (197, 122, 175)
                
                # eraser
                elif 892<x1<1087:
                    draw_color = (0,0,0)
            
            cv2.rectangle(img ,(x1,y1-25) ,(x2 ,y2+25) ,draw_color ,cv2.FILLED)
            
           
        
        # index finger up : drawing mode
        if fingers[1]==1 and fingers[2]==0 :
    
            if xp==0 and yp==0:
                xp,yp = x1,y1
            
            if draw_color==(0,0,0):
                cv2.line(img ,(xp,yp) ,(x1,y1) ,draw_color ,eraser_thickness)
                cv2.line(img_canvas ,(xp,yp) ,(x1,y1) ,draw_color ,eraser_thickness)
            
            else:
                cv2.line(img ,(xp,yp) ,(x1,y1) ,draw_color ,brush_thickness)
                cv2.line(img_canvas ,(xp,yp) ,(x1,y1) ,draw_color ,brush_thickness)
            
            xp,yp = x1,y1
        
    
    imgGray = cv2.cvtColor(img_canvas ,cv2.COLOR_BGR2GRAY)

    _ , imgInv = cv2.threshold(imgGray ,50 ,255 ,cv2.THRESH_BINARY_INV)
    
    imgInv = cv2.cvtColor(imgInv ,cv2.COLOR_GRAY2BGR)
    
   
    img = cv2.bitwise_and(img ,imgInv)
    img = cv2.bitwise_or(img ,img_canvas)
    
    img[0:141,0:1135] = header

    cv2.imshow('Painter' ,img)
    cv2.imshow('Painter1' ,img_canvas)
    cv2.imshow('Painte2' ,imgInv)
    
    if cv2.waitKey(1)&0xff == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()